### Python Implementation of Modern Portfolio Theory

This code relates to my article on Medium.

For any questions, please leave a comment on the article or here on Github.

In [ ]:
#!pip install -U kaleido

In [ ]:
import pandas as pd
import numpy as np
from yahoofinancials import YahooFinancials
import sqlite3
from tqdm import tqdm

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff

dbName = "stocks.db"
con = sqlite3.connect(dbName)
cur = con.cursor()

In [ ]:
stocktickers=['AAPL','MSFT','AMZN','TSLA','GOOG','BRK-B','UNH','JNJ','XOM','META','NVDA','JPM','PG','V','HD','CVX','MA','PFE','LLY','PEP']
numTradeDays = 252

def pullFromDatabase(tickers, startdate, enddate):
    sqlSelect = "SELECT ticker, date, dailyreturn FROM alltickers where ticker in ("
    for ticker in tickers:
        sqlSelect = sqlSelect+"\'" + ticker+"\',"
    sqlSelect = sqlSelect.strip(',')+ ") and date>=\'" + startdate + "\' and date<=\'" + enddate +"\'"
    res = cur.execute(sqlSelect)
    return res.fetchall()

def getData(stocktickers, startdate, enddate):
    database = pd.DataFrame(pullFromDatabase(stocktickers, startdate, enddate))
    database.columns = ['ticker','date','dailyreturn']
    daily_returns = pd.DataFrame(database.pivot(index='date', columns='ticker', values='dailyreturn'))
    daily_returns.head()
    daily_returns.reset_index(drop=True)
    return daily_returns


In [ ]:
def pullFromTreasury(startdate, enddate):
    con = sqlite3.connect(dbName)
    cur = con.cursor()
    sqlSelect = "SELECT avg(AdjClose) FROM treasury where date>=\'" + startdate + "\' and date<=\'" + enddate +"\'"
    res = cur.execute(sqlSelect)
    data = res.fetchall()
    cur.close()
    con.close()
    return data

def getRiskFreeRate(startdate, enddate):
    data = pullFromTreasury(startdate, enddate)
    return data[0][0] * 0.01

In [ ]:
#-- Get annualised mean returns
def getReturnsAndVariances(daily_returns):
    mus = (daily_returns.mean())*252

    #-- Get covariances
    #- Multiply by 252 to annualise it (square root time for volatility but no square root for variance)
    #- Note: 252 trading days in a year
    #- https://quant.stackexchange.com/questions/4753/annualized-covariance
    cov = daily_returns.cov()*252
    
    #single ticker std dev
    std = daily_returns.std()
    
    return mus, cov, std

In [ ]:
def getGARunRecommendation(enddate):
    sqlSelect = "select date, runid, epoch, chromosome, max(sharpe), risk from garesults where date=\'" + enddate + "\'"
    print(sqlSelect)
    res = cur.execute(sqlSelect)
    gaResults = res.fetchall()
    gaResultsDf = pd.DataFrame(gaResults)
    gaResultsDf.columns= ['date','runid','epoch', 'chromosome', 'sharpe','risk']
    sharpe = float(gaResultsDf['sharpe'])
    risk = float(gaResultsDf['risk'])
    returns = sharpe * risk
    print('return ', returns, 'risk ', risk, 'sharpe ' , sharpe)
    return gaResultsDf['chromosome'], returns, risk

In [ ]:

def createRandomPortfolios(daily_returns, maxStocks, n_portfolios, mus, cov, std, risk_free_rate):

    #- How many assests to include in each portfolio
    n_assets = maxStocks

    #-- Initialize empty list to store mean-variance pairs for plotting
    mean_variance_pairs = []
    best_index = 0
    best_sharpe = 0

    np.random.seed(75)
    #-- Loop through and generate lots of random portfolios
    for i in range(n_portfolios):
        #- Choose assets randomly without replacement
        assets = np.random.choice(list(daily_returns.columns), n_assets, replace=False)
        #- Choose weights randomly
        weights = np.random.rand(n_assets)
        #- Ensure weights sum to 1
        weights = weights/sum(weights)

        #-- Loop over asset pairs and compute portfolio return and variance
        #- https://quant.stackexchange.com/questions/43442/portfolio-variance-explanation-for-equation-investments-by-zvi-bodie
        portfolio_E_Return, portfolio_E_Variance= calcPortfolioReturnAndVariance(assets, weights, mus, cov, std)
        sharpe = (portfolio_E_Return-risk_free_rate)/(portfolio_E_Variance**0.5)
        if (sharpe > best_sharpe):
            best_sharpe = sharpe
            best_index = i
        #-- Add the mean/variance pairs to a list for plotting
        mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance, sharpe])
    return mean_variance_pairs, best_index




In [ ]:
def calcPortfolioReturnAndVariance(assets, weights, mus, cov, std):
    #-- Loop over asset pairs and compute portfolio return and variance
    #- https://quant.stackexchange.com/questions/43442/portfolio-variance-explanation-for-equation-investments-by-zvi-bodie
    portfolio_E_Variance = 0
    portfolio_E_Return = 0
    for i in range(len(assets)):
        portfolio_E_Return += (weights[i] * mus.loc[assets[i]])
        portfolio_E_Variance += (weights[i] * std.loc[assets[i]])**2 #should be ticker sd instead of return
        for j in range(len(assets)):
            #-- Add variance/covariance for each asset pair
            #- Note that when i==j this adds the variance
            portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
    return portfolio_E_Return, portfolio_E_Variance


In [ ]:

#-- Plot the risk vs. return of randomly generated portfolios
#-- Convert the list from before into an array for easy plotting
def plotRandomPortfolios(mean_variance_pairs, best_index, startdate, enddate):
    chromosome, returns, risk = getGARunRecommendation(enddate)
    mean_variance_pairs = np.array(mean_variance_pairs)
    rand_best = mean_variance_pairs[best_index,:]
    print('rand_best ', rand_best)


    fig = go.Figure()
    fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, y=mean_variance_pairs[:,0], 
                          marker=dict(color=(mean_variance_pairs[:,2]), 
                                      showscale=True, 
                                      size=7,
                                      line=dict(width=1),
                                      colorscale="RdBu",
                                      colorbar=dict(title="Sharpe<br>Ratio")
                                     ), 
                          mode='markers'))
    fig.add_trace(go.Scatter(x=[risk], y=[returns], marker_symbol='x',
                            marker=dict(color=(returns/risk), 
                                      showscale=False, 
                                      size=20,
                                      line=dict(width=1),
                                      colorscale="RdBu",
                                      colorbar=dict(title="Sharpe<br>Ratio")
                                     ), 
                          mode='markers'))
    fig.add_trace(go.Scatter(x=[rand_best[1]**0.5], y=[rand_best[0]], marker_symbol='star',
                            marker=dict(
                                      showscale=False, 
                                      size=20,
                                      line=dict(width=1),
                                     ), 
                          mode='markers'))
    fig.update_layout(template='plotly_white',
                      xaxis=dict(title='Annualised Risk (Volatility)'),
                      yaxis=dict(title='Annualised Return'),
                      title='Efficient Frontier ('+enddate+') GA Sharpe:'
                      +'{0:.2f}'.format(returns/risk) + ' Rand Best Sharpe: ' + '{0:.2f}'.format(rand_best[2]),
                      width=850,
                      height=500)

    
    fig.update_layout(coloraxis_colorbar=dict(title="Sharpe Ratio"))
    fig.show()
    fig.write_image('images/'+enddate+".png")
    
    # return, risk, sharpe for randbest and ga
    return [(rand_best[0], rand_best[1]**0.5, rand_best[2]), (returns, risk, returns/risk)]

In [ ]:
def computeAndPlot(stocktickers, startdate, enddate):
    daily_returns=getData(stocktickers, startdate, enddate)
    mus, cov, std = getReturnsAndVariances(daily_returns)
    risk_free_rate=getRiskFreeRate(startdate, enddate)
    mean_variance_pairs, best_index = createRandomPortfolios(daily_returns, 10, 1000, mus, cov, std, risk_free_rate)
    return plotRandomPortfolios(mean_variance_pairs, best_index, startdate, enddate)

In [ ]:
# run plots for everyday backwards from 2nd September, Friday
import datetime
import time

def dateToTimestamp(datestr):
    element = datetime.datetime.strptime(datestr,"%Y-%m-%d")
    return datetime.datetime.timestamp(element)
def timestampToDate(timestamp):
    return time.strftime("%Y-%m-%d", time.localtime(int(timestamp)))

def getDayOfWeek(datestr):
    return datetime.datetime.strptime(datestr,"%Y-%m-%d").strftime('%A')

runsStartdateTS = dateToTimestamp("2022-09-02") #more recent
runsEnddateTS = dateToTimestamp("2021-09-01") #less recent
startdate = "2022-09-02" #less recent
dataLengthInMonths = 12

logging = pd.DataFrame()

i=0 
while(runsStartdateTS > runsEnddateTS):
    enddate = timestampToDate(runsStartdateTS)
    startdate = timestampToDate(runsStartdateTS-(24*60*60*30*dataLengthInMonths))
    if (i>=0 and i<5):
        print("=============================== compute ", i, ' ', enddate, ' ', getDayOfWeek(enddate), ' startdate', startdate)
        best = computeAndPlot(stocktickers, startdate, enddate)
        logging = logging.append({'date':enddate,
                                  'randbest_return':best[0][0],
                                  'randbest_risk':best[0][1],
                                  'randbest_sharpe':best[0][2],
                                  'GAbest_return':best[1][0],
                                  'GAbest_risk':best[1][1],
                                  'GAbest_sharpe':best[1][2]}, ignore_index=True)
        i+=1
    elif (i==5):
        #print("it Sunday, mate! ", enddate)
        i+=1
    elif (i==6):
        #print("Saturrrday! ",enddate)
        i=0
    
    runsStartdateTS -= 24*60*60 #decrement by 1 day

print(logging)
#logging.columns = ['date','randbest_return','randbest_risk', 'randbest_sharpe', 'GAbest_return', 'GAbest_risk', 'GAbest_sharpe']
logging.to_csv("GA_v_EF.csv")